In [28]:
#git clone -b prune-experiment https://github.com/atikul-islam-sajib/TreeBasedModel.git
!git clone -b prune-experiment https://github.com/markusloecher/TreeSandBox.git

In [2]:
%cd TreeSandBox

/content/TreeBasedModel


In [28]:
!pip install -e . --verbose

In [4]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd
#import numpy as np
#from sklearn.metrics import accuracy_score
from TreeModelsFromScratch.RandomForest import RandomForest
from copy import deepcopy

In [5]:
# Generate synthetic dataset
X, y = make_classification(n_samples=100, n_features=20, n_informative=10, n_redundant=10, n_clusters_per_class=2, random_state=42)
X = pd.DataFrame(X)
y = pd.Series(y)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Instantiate RandomForest
rf = RandomForest(n_trees=2, max_depth=10, min_samples_split=20, min_samples_leaf=5,
                  n_feature="sqrt", bootstrap=True, oob=True, criterion="gini",
                  treetype="classification", random_state=42)

# Fit the model
rf.fit(X_train, y_train)


/content/TreeBasedModel/TreeModelsFromScratch/RandomForest.py:277: UserWarning: 25 out of 75 samples do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates. These samples were dropped before computing the oob_score
  warn(message)


Understand the tree structure first

In [6]:
dt =deepcopy(rf.trees[0])
rootNode = dt.node_list[0] #??
print(rootNode.samples)
print(rootNode.threshold)
print(rootNode.value)
rootNode.left.samples
rootNode.right.leaf_node
rootNode.right.samples
dt._get_y_for_node(rootNode.right)
rootNode.right.id
rootNode.left.id
dt.decision_paths


75
-0.6577327560706901
0


[(0, 1), (0, 2, 3), (0, 2, 4, 5), (0, 2, 4, 6)]

## Try a recursive function

In [7]:
def traverseTree(parentNode, numLeafs):
    if parentNode.leaf_node:
        print("leaf node", parentNode.id, "with", parentNode.samples, "samples")
        return numLeafs+1
    else:
        print("inner node", parentNode.id, "with", parentNode.samples, "samples")
        if parentNode.left != None:
            print("left turn")
            numLeafs = traverseTree(parentNode.left,numLeafs)
        if parentNode.right != None:
            print("right turn")
            numLeafs = traverseTree(parentNode.right, numLeafs)
        return numLeafs

nLeafs = traverseTree(rootNode,0)
print(nLeafs, "leaves in total")

inner node 0 with 75 samples
left turn
leaf node 1 with 20 samples
right turn
inner node 2 with 55 samples
left turn
leaf node 3 with 17 samples
right turn
inner node 4 with 38 samples
left turn
leaf node 5 with 5 samples
right turn
leaf node 6 with 33 samples
4 leaves in total


Now we use the same idea of a recursive function to prune:

In [27]:
import math

def pruneTree(parentNode, min_samples_leaf = 10):
    if parentNode.leaf_node:
        print("leaf node", parentNode.id, "with", parentNode.samples, "samples")
        return
    else:
        leftChild = parentNode.left
        rightChild = parentNode.right
        print("inner node", parentNode.id, "with", parentNode.samples, "samples and children:",leftChild.id, rightChild.id)
        if (leftChild.samples < min_samples_leaf) & (rightChild.samples < min_samples_leaf):#easiest case
            print('\033[1m' + "pruning both children", leftChild.id, "with", leftChild.samples, "samples", rightChild.id, "with", rightChild.samples, "samples" + '\033[0m')
            parentNode.leaf_node = True
            parentNode.left = None
            parentNode.right = None
        elif (leftChild.samples >= min_samples_leaf) & (rightChild.samples >= min_samples_leaf):#also easy
            print("left turn to ", leftChild.id)
            pruneTree(leftChild,min_samples_leaf)
            print("right turn to ", rightChild.id)
            pruneTree(rightChild,min_samples_leaf)
        elif leftChild.samples < min_samples_leaf:
            print('\033[1m' + "pruning left child", leftChild.id, "with", leftChild.samples, "samples" + '\033[0m')
            parentNode.left = None
            #reminder: left_idxs = np.argwhere(X_column <= split_thresh).flatten()
            parentNode.threshold = -math.inf#ideally one should remove this useless inner node
            print("right turn to ", rightChild.id)
            pruneTree(rightChild,min_samples_leaf)
        elif rightChild.samples < min_samples_leaf:
            print('\033[1m' + "pruning right child", rightChild.id, "with", rightChild.samples, "samples" + '\033[0m')
            parentNode.right = None
            parentNode.threshold = math.inf#ideally one should remove this useless inner node
            print("left turn to ", leftChild.id)
            pruneTree(leftChild,min_samples_leaf)

    #print("parent node", parentNode.id, "with", "children:",parentNode.left, parentNode.right)
    #if (parentNode.left == None) & (parentNode.right == None):
    #    parentNode.leaf_node = True

dt =deepcopy(rf.trees[0])
rootNode = dt.node_list[0] #??

pruneTree(rootNode,20)
print("------------ done pruning, now just traversing again:---------------")
nLeafs = traverseTree(rootNode,0)
print(nLeafs, "leaves in total")

inner node 0 with 75 samples and children: 1 2
left turn to  1
leaf node 1 with 20 samples
right turn to  2
inner node 2 with 55 samples and children: 3 4
pruning left child 3 with 17 samples
right turn to  4
inner node 4 with 38 samples and children: 5 6
pruning left child 5 with 5 samples
right turn to  6
leaf node 6 with 33 samples
------------ done pruning, now just traversing again:---------------
inner node 0 with 75 samples
left turn
leaf node 1 with 20 samples
right turn
inner node 2 with 55 samples
right turn
inner node 4 with 38 samples
right turn
leaf node 6 with 33 samples
2 leaves in total


While the routine above sort of works, it needs two major improvements:
1. The samples in the pruned nodes are discarded at the moment, whereas I think it would be better to redistribute them along all the children.
2. When only one child is pruned, the other surviving one becomes a useless inner node without a split. (I set the split threshold to infinity) So best would be to remove that node.

# **Task 1 & 2**

**Task 1**: Redistribute Samples from Pruned Nodes to All Remaining Children
1. **Function Creation:** Developed a function `redistribute_samples_to_children` to distribute samples from pruned nodes to their remaining sibling nodes.

2. **Integration with Pruning Logic:** Incorporated this function into the pruning process so that whenever a node is pruned, its samples are redistributed among the remaining children.

**Task 2**: Remove Useless Inner Nodes and Promote the Surviving Child
1. **Identifying Useless Inner Nodes:** Updated the pruning logic to detect when only one child remains after pruning.

2. **Promotion of Surviving Child:** Removed the useless inner node by promoting the surviving child, transferring its properties to the parent node.

3. **Recursive Pruning:** Ensured that the pruning process continues recursively if the promoted node is not a leaf node.

In [28]:
import math

def redistribute_samples_to_children(parentNode, prunedChild):
    remaining_children = [parentNode.left, parentNode.right]
    remaining_children = [child for child in remaining_children if child and child.id != prunedChild.id]

    if prunedChild and prunedChild.leaf_node and remaining_children:
        num_remaining_children = len(remaining_children)
        samples_per_child = prunedChild.samples // num_remaining_children
        for child in remaining_children:
            print(f"Redistributing {samples_per_child} samples from pruned node {prunedChild.id} to child node {child.id}")
            child.samples += samples_per_child

def pruneTree(parentNode, min_samples_leaf=10):
    if parentNode.leaf_node:
        print("leaf node", parentNode.id, "with", parentNode.samples, "samples")
        return
    else:
        leftChild = parentNode.left
        rightChild = parentNode.right
        print("inner node", parentNode.id, "with", parentNode.samples, "samples and children:", leftChild.id if leftChild else None, rightChild.id if rightChild else None)

        if leftChild and rightChild:
            if (leftChild.samples < min_samples_leaf) and (rightChild.samples < min_samples_leaf):
                # Prune both children
                print('\033[1m' + "Pruning both children", leftChild.id, "with", leftChild.samples, "samples", rightChild.id, "with", rightChild.samples, "samples" + '\033[0m')
                parentNode.leaf_node = True
                parentNode.left = None
                parentNode.right = None
                # Distribute samples from pruned nodes
                redistribute_samples_to_children(parentNode, leftChild)
                redistribute_samples_to_children(parentNode, rightChild)
            elif (leftChild.samples >= min_samples_leaf) and (rightChild.samples >= min_samples_leaf):
                # Recurse on both children
                print("left turn to ", leftChild.id)
                pruneTree(leftChild, min_samples_leaf)
                print("right turn to ", rightChild.id)
                pruneTree(rightChild, min_samples_leaf)
            elif leftChild.samples < min_samples_leaf:
                # Prune left child only
                print('\033[1m' + "Pruning left child", leftChild.id, "with", leftChild.samples, "samples" + '\033[0m')
                redistribute_samples_to_children(parentNode, leftChild)
                parentNode.left = None
                print(f"Removing useless inner node {parentNode.id} and promoting right child {rightChild.id}")
                # Remove useless inner node and promote the right child
                parentNode.left = rightChild.left
                parentNode.right = rightChild.right
                parentNode.threshold = rightChild.threshold
                parentNode.leaf_node = rightChild.leaf_node
                if not parentNode.leaf_node:
                    pruneTree(parentNode, min_samples_leaf)
            elif rightChild.samples < min_samples_leaf:
                # Prune right child only
                print('\033[1m' + "Pruning right child", rightChild.id, "with", rightChild.samples, "samples" + '\033[0m')
                redistribute_samples_to_children(parentNode, rightChild)
                parentNode.right = None
                print(f"Removing useless inner node {parentNode.id} and promoting left child {leftChild.id}")
                # Remove useless inner node and promote the left child
                parentNode.left = leftChild.left
                parentNode.right = leftChild.right
                parentNode.threshold = leftChild.threshold
                parentNode.leaf_node = leftChild.leaf_node
                if not parentNode.leaf_node:
                    pruneTree(parentNode, min_samples_leaf)

        elif leftChild:
            if leftChild.samples < min_samples_leaf:
                print('\033[1m' + "Pruning left child", leftChild.id, "with", leftChild.samples, "samples" + '\033[0m')
                parentNode.left = None
                redistribute_samples_to_children(parentNode, leftChild)
                parentNode.leaf_node = True

        elif rightChild:
            if rightChild.samples < min_samples_leaf:
                print('\033[1m' + "Pruning right child", rightChild.id, "with", rightChild.samples, "samples" + '\033[0m')
                parentNode.right = None
                redistribute_samples_to_children(parentNode, rightChild)
                parentNode.leaf_node = True

if __name__ == "__main__":
  dt = deepcopy(rf.trees[0])
  rootNode = dt.node_list[0]

  pruneTree(rootNode, 20)
  print("------------ done pruning, now just traversing again:---------------")
  nLeafs = traverseTree(rootNode, 0)
  print(nLeafs, "leaves in total")

inner node 0 with 75 samples and children: 1 2
left turn to  1
leaf node 1 with 20 samples
right turn to  2
inner node 2 with 55 samples and children: 3 4
Pruning left child 3 with 17 samples
Redistributing 17 samples from pruned node 3 to child node 4
Removing useless inner node 2 and promoting right child 4
inner node 2 with 55 samples and children: 5 6
Pruning left child 5 with 5 samples
Redistributing 5 samples from pruned node 5 to child node 6
Removing useless inner node 2 and promoting right child 6
------------ done pruning, now just traversing again:---------------
inner node 0 with 75 samples
left turn
leaf node 1 with 20 samples
right turn
leaf node 2 with 55 samples
2 leaves in total
